In [18]:
import os
from neo4j import GraphDatabase

In [2]:
# Replace with your Neo4j connection details
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASS = os.getenv("NEO4J_PASS", "")

In [9]:
driver = GraphDatabase.driver(NEO4J_URI, auth=("neo4j", "password"))

In [13]:
def get_person_details_from_neo4j(person_name: str):
    """
    Fetch details of a broker (Person node) from Neo4j.
    Returns all properties stored in the Person node.
    """
    query = """
    MATCH (p:Person)
    WHERE p.name = $name
    RETURN p
    """

    with driver.session() as session:
        result = session.run(query, name=person_name)
        record = result.single()
        if not record:
            return {"broker": None}

        person_node = record["p"]
        # convert Node properties to dict
        return {"broker": dict(person_node)}

In [15]:
get_person_details_from_neo4j("Mie Kim")

{'broker': {'role': "Seller's Representative",
  'name': 'Mie Kim',
  'title': 'Associate, Coldwell Banker',
  'url': '/people/mie-kim'}}

In [ ]:
"""Subject: Bridge Loan Opportunity
Hi Sabrina,
I wanted to share an off-market bridge opportunity that could be a great fit. The sponsor is experienced, the asset is cash flowing, and only a select group of lenders are reviewing this deal.
Shawna Menifee is seeking a $4.2M bridge loan to refinance existing debt on a property at 98–100 Bedford Avenue, Brooklyn, NY. The sponsor is targeting a 30-month loan term to complete renovations and full stabilization, with plans to refinance into long-term permanent debt once the repositioning is complete. He’s currently seeking pricing in the range of SOFR + 530 bps, depending on structure and recourse.
The building is 95% occupied, with residential rents slightly below market, offering near-term upside. Trader Joe’s anchors the retail portion under a long-term lease with annual escalations, providing steady cash flow. Planned upgrades include unit renovations and common-area improvements, which are expected to take approximately 24–28 months, after which the property is projected to achieve a stabilized value of around $38M.
Menifee brings 30+ years of NYC real estate experience and has developed and managed over $1.2B in multifamily and mixed-use assets.
 www.reynoldsproperties.com/about
Let me know a good time to connect to discuss this opportunity.
Best,
Bryan Son

"""

'Subject: Bridge Loan Opportunity\nHi Sabrina,\nI wanted to share an off-market bridge opportunity that could be a great fit. The sponsor is experienced, the asset is cash flowing, and only a select group of lenders are reviewing this deal.\nMie Kim is seeking a $4.2M bridge loan to refinance existing debt on a property at 98–100 Bedford Avenue, Brooklyn, NY. The sponsor is targeting a 30-month loan term to complete renovations and full stabilization, with plans to refinance into long-term permanent debt once the repositioning is complete. He’s currently seeking pricing in the range of SOFR + 530 bps, depending on structure and recourse.\nThe building is 95% occupied, with residential rents slightly below market, offering near-term upside. Trader Joe’s anchors the retail portion under a long-term lease with annual escalations, providing steady cash flow. Planned upgrades include unit renovations and common-area improvements, which are expected to take approximately 24–28 months, after 

In [ ]:
def fetch_broker_deals(broker_url: str):
    """
    Returns a list of deals a broker is involved in with all relevant info
    """
    query = """
    MATCH (p:Person {url: $broker_url})-[:PARTICIPATED_IN]->(d:Deal)
  OPTIONAL MATCH (d)-[:INVOLVES]->(prop:Property)
    OPTIONAL MATCH (p)-[:WORKS_FOR]->(org:Organization)
    RETURN d, 
           prop.address AS property_address, prop.url AS property_url,
           org.name AS organization_name, org.url AS organization_url, org.type as organization_type
    ORDER BY d.date DESC
    """
    with driver.session() as session:
        result = session.run(query, broker_url=broker_url)
        deals = []
        for record in result:
            deals.append({
                "deal_info": dict(record["d"]),
                "property_address": record["property_address"],
                "property_url": record["property_url"],
                "organization_name": record["organization_name"],
                "organization_url": record["organization_url"],
                "organization_type": record["organization_type"]
            })
    return (deals)

In [34]:
deals = fetch_broker_deals("/people/shawna-menifee")

{'d': <Node element_id='4:21c74eca-24d2-4b9b-a273-ff06f72301a3:13362' labels=frozenset({'Deal'}) properties={'date': '12/02/2021', 'floors': '1', 'square feet': '14,114', 'term years': '12', 'property': '477 Madison Ave', 'type': 'Lease', 'price per square foot': 'N/A', 'url': '/activity/477-madison-ave-lease-12022021-ariel-alternatives'}>, 'property_address': '477 Madison Ave, New York, NY, 10022', 'property_url': '/buildings/477-madison-ave-new-york-ny', 'organization_name': 'JLL', 'organization_url': '/organizations/jll', 'organization_type': 'Diversified'}
{'d': <Node element_id='4:21c74eca-24d2-4b9b-a273-ff06f72301a3:13362' labels=frozenset({'Deal'}) properties={'date': '12/02/2021', 'floors': '1', 'square feet': '14,114', 'term years': '12', 'property': '477 Madison Ave', 'type': 'Lease', 'price per square foot': 'N/A', 'url': '/activity/477-madison-ave-lease-12022021-ariel-alternatives'}>, 'property_address': '477 Madison Ave, New York, NY, 10022', 'property_url': '/buildings/47

# Test Agent

In [3]:
import requests
import json

In [31]:
url = "http://localhost:8002/run"

In [10]:
def create_session(uid, sid):
    url = f"http://localhost:8000/apps/intellj_agent/users/{uid}/sessions/{sid}"
    
    payload = {}
    headers = {
        "Content-Type": "application/json"
        }

    response = requests.post(url, headers=headers, json=payload)

    print(response.status_code)
    print(response.json())  # or response.text if it's not JSON


In [12]:
uid = "u_123"
sid = "s_123"
create_session(uid, sid)

409
{'detail': 'Session already exists: s_123'}


In [28]:
payload = {
    "appName": "intellj_agent",
    "userId": uid,
    "sessionId": sid,
    "newMessage": {
        "role": "user",
        "parts": [
            {"text": "Marc Peeler"},
        ]
    },
    "streaming": False
}

headers = {
    "Content-Type": "application/json"
}

In [36]:
response = requests.post(url, headers=headers, data=json.dumps(payload))

print(response.status_code)
print(response.json())

200
[{'modelVersion': 'gemini-2.5-flash', 'content': {'parts': [{'functionCall': {'id': 'adk-10c74455-7c78-4c54-b59f-9b324c105219', 'args': {'names': ['Marc Peeler']}, 'name': 'normalize_names'}}], 'role': 'model'}, 'finishReason': 'STOP', 'usageMetadata': {'candidatesTokenCount': 7, 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 7}], 'promptTokenCount': 6119, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 6119}], 'thoughtsTokenCount': 178, 'totalTokenCount': 6304, 'trafficType': 'ON_DEMAND'}, 'avgLogprobs': -5.898523058210101, 'invocationId': 'e-e4354559-b583-4e8b-9c5a-bf2e8ef7dddf', 'author': 'BrokerExtractionAgent', 'actions': {'stateDelta': {}, 'artifactDelta': {}, 'requestedAuthConfigs': {}, 'requestedToolConfirmations': {}}, 'longRunningToolIds': [], 'id': 'cbb1836c-83c2-4aaa-ba56-aa221e5486c2', 'timestamp': 1772108353.616672}, {'content': {'parts': [{'functionResponse': {'id': 'adk-10c74455-7c78-4c54-b59f-9b324c105219', 'name': 'normalize_names', 'res

In [37]:
res = response.json()

In [38]:
print(res[-1]['content']['parts'][0]['text'])

Marc Peeler was not found in our database. The information provided below is obtained from an internet search.

## Broker Profile: **Marc Peeler**

### 🏢 Organizations & Locations
*   **Current Organization:** Partners Real Estate
    *   **Role:** Partner, specializing in Industrial and Retail Services.
*   **Previous Organizations:**
    *   NewQuest's Investment Sales Division (Senior Vice President)
    *   Stan Johnson Company (Associate Director of Investment Sales)
*   **Active Locations:** Houston, TX; Dallas/Ft Worth; Seattle/Puget Sound; St. Louis; San Antonio; Minneapolis/St Paul; Birmingham; Bakersfield; El Paso.
*   **Organizational Involvement:** International Council of Shopping Centers (ICSC).

### 📊 Deal Highlights
*   **Total Volume:** In excess of $1 billion.
*   **Details:**
    *   Transacted over $600 million in net lease assets while at Stan Johnson Company.
    *   Closed $10 Million across 15 deals, with the largest being a $10,000,000 sale for 2430 Farrell Roa